In [8]:
import pandas as pd
import json
from glob import glob
import chardet

In [9]:
outputDir = glob("../01_GIS/sensitivity/csvs/*")
outputDir

['../01_GIS/sensitivity/csvs\\parisSensitivity1000Output.csv',
 '../01_GIS/sensitivity/csvs\\parisSensitivity1500Output.csv',
 '../01_GIS/sensitivity/csvs\\parisSensitivity2000Output.csv',
 '../01_GIS/sensitivity/csvs\\parisSensitivity3000Output.csv',
 '../01_GIS/sensitivity/csvs\\parisSensitivity500Output.csv',
 '../01_GIS/sensitivity/csvs\\vancouverSensitivity1000Output.csv',
 '../01_GIS/sensitivity/csvs\\vancouverSensitivity1500Output.csv',
 '../01_GIS/sensitivity/csvs\\vancouverSensitivity2000Output.csv',
 '../01_GIS/sensitivity/csvs\\vancouverSensitivity3000Output.csv',
 '../01_GIS/sensitivity/csvs\\vancouverSensitivity500Output.csv']

In [10]:
outputDir[0].split('/')[-1][5:-4]

'parisSensitivity1000Output'

In [11]:
dfs = {}

for output in outputDir:
    dfs[f"{output.split('/')[-1][5:-4]}"] = pd.read_csv(output)

In [12]:
keys = list(dfs.keys())

In [13]:
dfs[f"{keys[0]}"].columns

Index(['left', 'bottom', 'right', 'top', 'fid', 'Unnamed: 0_x', 'index_x',
       'Aesculus Carnea', 'Platanus Hispanica', 'Platanus Orientalis',
       ...
       'Pittosporaceae', 'Pittosporaceae PCT', 'Hydrangeaceae',
       'Hydrangeaceae PCT', 'Garryaceae', 'Podocarpaceae', 'Sciadopityaceae',
       'Garryaceae PCT', 'Podocarpaceae PCT', 'Sciadopityaceae PCT'],
      dtype='object', length=1966)

In [14]:
keys[0].split("Sens")[0]

'paris'

In [15]:
mergeDf = []

for i in range(0, len(keys)):
    tempDf = dfs[f"{keys[i]}"][["sum","num","speciesDiversity"]]
    tempDf.columns = ["sum","num","shannon"]
    tempDf['scale'] = int(keys[i].split('y')[1][:-6])
    tempDf["taxa"] = "species"
    tempDf["city"] = keys[i].split("Sens")[0]
    if (len(mergeDf)>0):
        mergeDf=pd.concat([mergeDf,tempDf],axis=0)
    else:
        mergeDf=tempDf
        
    tempDf = dfs[f"{keys[i]}"][["sum","num","genusDiversity"]]
    tempDf.columns = ["sum","num","shannon"]
    tempDf['scale'] = int(keys[i].split('y')[1][:-6])
    tempDf["taxa"] = "genus"
    tempDf["city"] = keys[i].split("Sens")[0]
    mergeDf=pd.concat([mergeDf,tempDf],axis=0)
    
    tempDf = dfs[f"{keys[i]}"][["sum","num","familyDiversity"]]
    tempDf.columns = ["sum","num","shannon"]
    tempDf['scale'] = int(keys[i].split('y')[1][:-6])
    tempDf["taxa"] = "family"
    tempDf["city"] = keys[i].split("Sens")[0]
    mergeDf=pd.concat([mergeDf,tempDf],axis=0)

<ipython-input-15-1be6b8ffb183>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempDf['scale'] = int(keys[i].split('y')[1][:-6])
<ipython-input-15-1be6b8ffb183>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempDf["taxa"] = "species"
<ipython-input-15-1be6b8ffb183>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

In [16]:
mergeDf.to_csv("./SensitivityCombined.csv")

In [27]:
mergeDf[(mergeDf['city']=="paris")&(mergeDf['scale']==3000)]['shannon'].mean()

1.809690394686844

In [31]:
cities = ["paris","vancouver"]
scales = [500,1000,1500,2000,3000]
taxas = ["species","genus","family"]

summaryData = []

for city in cities:
    for scale in scales:
        for taxa in taxas:
            
            tempDf = mergeDf[(mergeDf['city']==city)&(mergeDf['scale']==scale)&(mergeDf['taxa']==taxa)]
            tempObj = {
                'city': city,
                'scale': scale,
                'taxa': taxa,
                'mean': tempDf["shannon"].mean(),
                'sd': tempDf["shannon"].std(),
                'q25': tempDf["shannon"].quantile(.25),
                'q75': tempDf["shannon"].quantile(.75)
            }
            summaryData.append(tempObj)

In [35]:
pd.DataFrame(summaryData)

,city,scale,taxa,mean,sd,q25,q75
0,paris,500,species,1.598780,0.804658,1.034874,2.230975
1,paris,500,genus,1.373408,0.723996,0.853990,1.947345
2,paris,500,family,1.236983,0.626290,0.807526,1.720567
3,paris,1000,species,1.962765,0.823547,1.530121,2.576654
4,paris,1000,genus,1.658221,0.715278,1.251394,2.180730
5,paris,1000,family,1.473664,0.608447,1.160705,1.947575
6,paris,1500,species,2.091487,0.979848,1.791047,2.772391
7,paris,1500,genus,1.741487,0.809914,1.526505,2.282555
8,paris,1500,family,1.526212,0.697305,1.365767,2.022857
9,paris,2000,species,2.034677,1.014505,1.042574,2.827733
